In [43]:
import re
import pandas as pd
from collections import defaultdict
import operator
from pprint import pprint

# Set Pandas to display all rows of dataframes
pd.set_option('display.max_rows', 500)

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [44]:
sections = pd.read_csv('sections.csv')

In [45]:
words = pd.read_csv('words.csv')
words.head()

,Unnamed: 0,word_num,word,sentence_num,section_num,book_title,book_num,lemma,POS,stop
0,0,1,I,1,1,ADI PARVA,1,-PRON-,PRON,True
1,1,1,Om,2,1,ADI PARVA,1,Om,PROPN,False
2,2,2,!,2,1,ADI PARVA,1,!,PUNCT,False
3,3,1,Having,3,1,ADI PARVA,1,have,VERB,False
4,4,2,bowed,3,1,ADI PARVA,1,bow,VERB,False


In [46]:
bookTitles = []
for i in range(len(sections)):
    bookTitles.append(sections.loc[i]['book_title'])
bookTitles = (set(bookTitles))
bookTitles = list(bookTitles)
bookTitles

['UDYOGA PARVA',
 'MAUSALA-PARVA',
 'VANA PARVA',
 'SAUPTIKA-PARVA',
 'STRI-PARVA',
 'ASWAMEDHA PARVA',
 'MAHAPRASTHANIKA PARVA',
 'BHISHMA PARVA',
 'DRONA PARVA',
 'KARNA-PARVA',
 'ANUSASANA PARVA',
 'ADI PARVA',
 'SVARGAROHANIKA-PARVA',
 'SHALYA-PARVA',
 'SANTI PARVA',
 'ASRAMAVASIKA PARVA',
 'VIRATA PARVA',
 'SABHA PARVA']

In [47]:
mb = {}
for i in range(len(bookTitles)):
    chaptersDict = {}
    for j in range(len(sections)):
        if bookTitles[i] is sections.loc[j]['book_title']:
            chapterNum = "Chapter"+str(sections.loc[j]['section_num'])
            chaptersDict[chapterNum] = (sections.loc[j]['section_num'],sections.loc[j]['section'])
    mb[bookTitles[i]] = chaptersDict

In [48]:
print(mb['ADI PARVA'])

{'Chapter1': (1, ' I\n\nOm! Having bowed down to Narayana and Nara, the most exalted male being,\nand also to the goddess Saraswati, must the word Jaya be uttered.\n\nUgrasrava, the son of Lomaharshana, surnamed Sauti, well-versed in the\nPuranas, bending with humility, one day approached the great sages of\nrigid vows, sitting at their ease, who had attended the twelve years\'\nsacrifice of Saunaka, surnamed Kulapati, in the forest of Naimisha. Those\nascetics, wishing to hear his wonderful narrations, presently began to\naddress him who had thus arrived at that recluse abode of the inhabitants\nof the forest of Naimisha. Having been entertained with due respect by\nthose holy men, he saluted those Munis (sages) with joined palms, even\nall of them, and inquired about the progress of their asceticism. Then\nall the ascetics being again seated, the son of Lomaharshana humbly\noccupied the seat that was assigned to him. Seeing that he was\ncomfortably seated, and recovered from fatigue,

In [49]:
# Word Count
for book in mb:
    print('{:,} words in {}'.format(sum(len(mb[book][chapter][1].split()) for chapter in mb[book]), book))
print()    
print('{:,} total words in collection'.format(sum(len(mb[book][chapter][1].split())
                                                for book in mb
                                                for chapter in mb[book])))

189,966 words in UDYOGA PARVA
8,083 words in MAUSALA-PARVA
326,211 words in VANA PARVA
21,354 words in SAUPTIKA-PARVA
21,840 words in STRI-PARVA
89,251 words in ASWAMEDHA PARVA
3,067 words in MAHAPRASTHANIKA PARVA
161,404 words in BHISHMA PARVA
251,461 words in DRONA PARVA
138,259 words in KARNA-PARVA
299,693 words in ANUSASANA PARVA
228,537 words in ADI PARVA
8,645 words in SVARGAROHANIKA-PARVA
91,591 words in SHALYA-PARVA
536,274 words in SANTI PARVA
32,697 words in ASRAMAVASIKA PARVA
62,847 words in VIRATA PARVA
77,008 words in SABHA PARVA

2,548,188 total words in collection


In [50]:
# Average word length
for book in mb:
    text = ''
    for chapter in mb[book]:
        text = text + mb[book][chapter][1]
    print('{:.2f} Average word length in {}'.format(len(text) / len(text.split()), book))

5.82 Average word length in UDYOGA PARVA
5.83 Average word length in MAUSALA-PARVA
5.77 Average word length in VANA PARVA
5.83 Average word length in SAUPTIKA-PARVA
5.82 Average word length in STRI-PARVA
5.92 Average word length in ASWAMEDHA PARVA
5.80 Average word length in MAHAPRASTHANIKA PARVA
5.98 Average word length in BHISHMA PARVA
5.96 Average word length in DRONA PARVA
5.91 Average word length in KARNA-PARVA
5.73 Average word length in ANUSASANA PARVA
5.83 Average word length in ADI PARVA
5.87 Average word length in SVARGAROHANIKA-PARVA
5.92 Average word length in SHALYA-PARVA
5.88 Average word length in SANTI PARVA
5.88 Average word length in ASRAMAVASIKA PARVA
5.83 Average word length in VIRATA PARVA
5.92 Average word length in SABHA PARVA


In [51]:
# Chapters in books
for book in mb:
    chapters = 0
    for chapter in mb[book]:
        chapters += 1
    print('{} chapters in {}'.format(chapters, book))

199 chapters in UDYOGA PARVA
8 chapters in MAUSALA-PARVA
313 chapters in VANA PARVA
18 chapters in SAUPTIKA-PARVA
27 chapters in STRI-PARVA
92 chapters in ASWAMEDHA PARVA
3 chapters in MAHAPRASTHANIKA PARVA
124 chapters in BHISHMA PARVA
200 chapters in DRONA PARVA
96 chapters in KARNA-PARVA
168 chapters in ANUSASANA PARVA
236 chapters in ADI PARVA
6 chapters in SVARGAROHANIKA-PARVA
65 chapters in SHALYA-PARVA
363 chapters in SANTI PARVA
39 chapters in ASRAMAVASIKA PARVA
72 chapters in VIRATA PARVA
80 chapters in SABHA PARVA


In [52]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [53]:
data = [mb[book][chapter][1].replace('\n', '') for book in mb for chapter in mb[book]]
print(data[0])

 IOM! HAVING BOWED down to Narayana, and Nara the most exalted of malebeings, and also to the goddess Saraswati, must the word Jaya be uttered."Vaisampayana said, 'Then those valiant descendants of Kuru, who belongedto the same party (with Virata), having joyfully celebrated the nuptialsof Abhimanyu and rested themselves that night, presented themselves atdawn, well pleased, in the court of Virata, And the chamber of the kingof the Matsya was full of riches, and variegated with choice gems andprecious stones, with seats methodically arranged, adorned with garlands,and filled with fragrance. And those mighty monarchs of men all came tothat place, And on the seats in front sat the two kings Virata andDrupada. And the revered and aged rulers of the earth, and Valarama andKrishna along with their father, all sat there. And close to the king ofPanchala was seated the great hero of the race of Sini, together with theson of Rohini. And side by side with the king of the Matsya sat Krishnaand Y

In [54]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))
print(len(data_words))
print(len(data_words[:1][0]))
print(data_words[:1])

2109
822
[['iom', 'having', 'bowed', 'down', 'to', 'narayana', 'and', 'nara', 'the', 'most', 'exalted', 'of', 'malebeings', 'and', 'also', 'to', 'the', 'goddess', 'saraswati', 'must', 'the', 'word', 'jaya', 'be', 'uttered', 'vaisampayana', 'said', 'then', 'those', 'valiant', 'descendants', 'of', 'kuru', 'who', 'belongedto', 'the', 'same', 'party', 'with', 'virata', 'having', 'joyfully', 'celebrated', 'the', 'nuptialsof', 'abhimanyu', 'and', 'rested', 'themselves', 'that', 'night', 'presented', 'themselves', 'atdawn', 'well', 'pleased', 'in', 'the', 'court', 'of', 'virata', 'and', 'the', 'chamber', 'of', 'the', 'kingof', 'the', 'matsya', 'was', 'full', 'of', 'riches', 'and', 'variegated', 'with', 'choice', 'gems', 'andprecious', 'stones', 'with', 'seats', 'methodically', 'arranged', 'adorned', 'with', 'garlands', 'and', 'filled', 'with', 'fragrance', 'and', 'those', 'mighty', 'monarchs', 'of', 'men', 'all', 'came', 'tothat', 'place', 'and', 'on', 'the', 'seats', 'in', 'front', 'sat', 't

In [55]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  


# Faster way to get a sentence clubbed as a trigram/bigram
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['iom', 'having', 'bowed_down', 'to', 'narayana', 'and', 'nara', 'the', 'most_exalted', 'of', 'malebeings', 'and', 'also', 'to', 'the', 'goddess_saraswati_must', 'the', 'word_jaya', 'be', 'uttered', 'vaisampayana', 'said', 'then', 'those', 'valiant', 'descendants', 'of', 'kuru', 'who', 'belongedto', 'the', 'same', 'party', 'with', 'virata', 'having', 'joyfully', 'celebrated', 'the', 'nuptialsof', 'abhimanyu', 'and', 'rested', 'themselves', 'that', 'night', 'presented', 'themselves', 'atdawn', 'well_pleased', 'in', 'the', 'court', 'of', 'virata', 'and', 'the', 'chamber', 'of', 'the', 'kingof', 'the', 'matsya', 'was', 'full', 'of', 'riches', 'and', 'variegated', 'with', 'choice', 'gems', 'andprecious', 'stones', 'with', 'seats', 'methodically', 'arranged', 'adorned', 'with', 'garlands', 'and', 'filled', 'with', 'fragrance', 'and', 'those', 'mighty', 'monarchs', 'of', 'men', 'all', 'came', 'tothat', 'place', 'and', 'on', 'the', 'seats', 'in', 'front', 'sat', 'the', 'two', 'kings', 'virata

In [56]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc])# if token.pos_ in allowed_postags])
    return texts_out

In [57]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['iom', 'bow', 'narayana', 'nara', 'exalted', 'malebeing', 'also', 'goddess_saraswati', 'must', 'word_jaya', 'uttered', 'vaisampayana', 'say', 'valiant', 'descendant', 'kuru', 'belongedto', 'party', 'virata', 'joyfully', 'celebrate', 'nuptialsof', 'abhimanyu', 'rest', 'night', 'present', 'atdawn', 'well_pleased', 'court', 'virata', 'chamber', 'kingof', 'matsya', 'full', 'rich', 'variegate', 'choice', 'gem', 'andprecious', 'stone', 'seat', 'methodically', 'arrange', 'adorn', 'garland', 'fill', 'fragrance', 'mighty', 'monarchs', 'man', 'come', 'tothat', 'place', 'seat', 'front', 'sit', 'two', 'king', 'virata_anddrupada', 'revere', 'aged', 'ruler', 'earth', 'valarama', 'andkrishna', 'along', 'father', 'sit', 'close', 'king', 'ofpanchala', 'seat', 'great', 'hero', 'race', 'sini', 'together', 'theson', 'rohini', 'side', 'side', 'king', 'matsya', 'sit', 'krishnaand', 'yudhishthira', 'sons', 'king', 'drupada', 'bhima', 'arjuna', 'two', 'son', 'madri', 'pradyumna', 'samva', 'valiant', 'inbatt

In [58]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 3), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 2), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 2), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 3), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 2), (58, 1), (59, 1), (60, 3), (61, 1), (62, 1), (63, 2), (64, 1), (65, 1), (66, 1), (67, 2), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 3), (74, 1), (75, 2), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 3), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 1), (106, 2), (107, 1), (108, 1), (109, 1), (110, 1)

In [59]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('abhimanyu', 2),
  ('able', 1),
  ('accept', 1),
  ('accomplish', 1),
  ('accordingly', 1),
  ('address', 1),
  ('adorn', 1),
  ('affair', 1),
  ('aged', 1),
  ('along', 1),
  ('also', 3),
  ('ambassador', 1),
  ('andhow', 1),
  ('andkrishna', 1),
  ('andof', 1),
  ('andprecious', 1),
  ('andprowess', 1),
  ('another', 1),
  ('approbation', 1),
  ('arjuna', 2),
  ('arrange', 1),
  ('asingle', 1),
  ('ask', 1),
  ('assemble', 1),
  ('assembled_together', 1),
  ('assembly', 2),
  ('atdawn', 1),
  ('attention', 1),
  ('band', 1),
  ('battle', 1),
  ('beautiful', 1),
  ('befor', 1),
  ('belongedto', 1),
  ('beseech', 1),
  ('beseechthem', 1),
  ('best', 1),
  ('bestow', 1),
  ('bhima', 1),
  ('birth', 1),
  ('bow', 1),
  ('boy', 1),
  ('brave', 1),
  ('brotherthen', 1),
  ('capable', 2),
  ('celebrate', 1),
  ('celestial', 1),
  ('chamber', 1),
  ('choice', 1),
  ('close', 1),
  ('come', 1),
  ('consider', 3),
  ('consistent', 1),
  ('consult', 1),
  ('court', 1),
  ('covet', 1),
  ('cr

In [60]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           passes=10)

In [66]:
# Print the Keyword in the 20 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.000*"thou" + 0.000*"art" + 0.000*"king" + 0.000*"one" + 0.000*"son" + '
  '0.000*"great" + 0.000*"thy" + 0.000*"say" + 0.000*"thee" + 0.000*"like"'),
 (1,
  '0.019*"thou" + 0.018*"thy" + 0.015*"one" + 0.011*"man" + 0.010*"king" + '
  '0.008*"thee" + 0.007*"may" + 0.007*"say" + 0.007*"act" + 0.006*"foe"'),
 (2,
  '0.022*"country" + 0.018*"hero" + 0.017*"king" + 0.015*"subjugate" + '
  '0.014*"sway" + 0.014*"son" + 0.013*"pandu" + 0.013*"vanquish" + '
  '0.011*"march" + 0.011*"call"'),
 (3,
  '0.001*"lean" + 0.000*"artpale" + 0.000*"greatest" + 0.000*"theethese" + '
  '0.000*"uprightness" + 0.000*"grandeur" + 0.000*"hispride" + 0.000*"toanger" '
  '+ 0.000*"aselfish" + 0.000*"theseabstruse"'),
 (4,
  '0.025*"king" + 0.014*"gold" + 0.012*"thousand" + 0.011*"unto" + '
  '0.011*"bring" + 0.011*"yudhishthira" + 0.010*"give" + 0.009*"tribute" + '
  '0.008*"gate" + 0.007*"also"'),
 (5,
  '0.046*"thou" + 0.021*"art" + 0.017*"thee" + 0.015*"thy" + 0.011*"say" + '
  '0.010*"god" + 0.007

In [62]:
# Compute Perplexity
print ('Perplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print ('Coherence Score: ', coherence_lda)

Perplexity:  -9.794832799647246
Coherence Score:  0.43015249530674754


## The output of the cell below has been saved in a file 'lda.html'

In [67]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
pyLDAvis.display(vis)

In [68]:
pyLDAvis.save_html(vis, 'lda2.html')